# Live coding

Expected learning outcomes:

-class construction for specific data structure (graph class)

-module exploration (in particular networkx)

In [ ]:
import networkx as nx
import numpy as np
from collections import Counter

In [ ]:
print(nx.__version__)
nx.__file__

2.5


'/usr/local/lib/python3.6/dist-packages/networkx/__init__.py'

C:\\Users\\Johan\\Anaconda3\\lib\\site-packages\\numpy\\__init__.py'

In [ ]:
dir()

## Graph Class

We now integrate our prefered graph representation (adjacency dict) into a class that we can build on. For now we provide it with just placeholders for our data

In [ ]:
class Graph:
    def __init__(self, directed=False):
        self._nodes = {}
        self._edges = {}
        self._directed = directed


In [ ]:
def add_method(cls):
    def decorator(func):
        setattr(cls, func.__name__, func)
        return func
    return decorator

In [ ]:
?dir

In [ ]:
G = nx.DiGraph()

In [ ]:
#dir(G)

## Nodes

In [ ]:
@add_method(Graph)
def add_node(self, node, **kwargs):
    self._nodes[node] = kwargs


In [ ]:
@add_method(Graph)
def add_nodes_from(self, nodes, **kwargs):
    for node in nodes:
        if isinstance(node, tuple):
            self._nodes[node[0]] = node[1:]
        else:
            self._nodes[node] = kwargs

In [ ]:
G = Graph()

In [ ]:
isinstance(5,str)

False

In [ ]:
G.add_node("Nah", eye_color='brown')

In [ ]:
G._nodes

{'Nah': {'eye_color': 'brown'},
 'Noah': {'eye_color': 'brown', 'hair_color': 'blond', 'height': 193}}

In [ ]:
help(nx.Graph.add_node)

In [ ]:
G.add_nodes_from("ABC", color='red')

In [ ]:
G._nodes

{'A': {'color': 'red'}, 'B': {'color': 'red'}, 'C': {'color': 'red'}}

## Edges

In [ ]:
@add_method(Graph)
def add_edge(self, node_i, node_j, **kwargs):
    if node_i not in self._nodes:
        self.add_node(node_i)

    if node_j not in self._nodes:
        self.add_node(node_j)
    
    if node_i not in self._edges:
        self._edges[node_i] = {}
    
    if node_j not in self._edges[node_i]:
        self._edges[node_i][node_j] = {}

    self._edges[node_i][node_j] = kwargs
    
    if not self._directed:
        if node_j not in self._edges:
            self._edges[node_j] = {}

        if node_i not in self._edges[node_j]:
            self._edges[node_j][node_i] = {}

        self._edges[node_j][node_i] = kwargs


@add_method(Graph)
def add_edges_from(self, edges, **kwargs):
    for edge in edges:
        self.add_edge(*edge, **kwargs)
    


# edge = (node_i, node_j)

In [ ]:
edge_list = [
    ('A', 'B'),
    ('A', 'C'),
    ('A', 'E'),
    ('B', 'C'),
    ('C', 'D'),
    ('C', 'E'),
    ('D', 'E')]

In [ ]:
G = Graph()

In [ ]:
G.add_edges_from(edge_list)

In [ ]:
G._edges

{'A': {'B': {}, 'C': {}, 'E': {}},
 'B': {'A': {}, 'C': {}},
 'C': {'A': {}, 'B': {}, 'D': {}, 'E': {}},
 'D': {'C': {}, 'E': {}},
 'E': {'A': {}, 'C': {}, 'D': {}}}

In [ ]:
G._nodes

{'A': {}, 'B': {}, 'C': {}, 'D': {}, 'E': {}}

## Attribute methods

In [ ]:
@add_method(Graph)
def edgelist(self):
    e = []
    
    for node_i in self._edges:
        for node_j in self._edges[node_i]:
            e.append([node_i, node_j, self._edges[node_i][node_j]])
            
    return e 


In [ ]:
G.edgelist()

[['A', 'B', {}],
 ['A', 'C', {}],
 ['A', 'E', {}],
 ['B', 'A', {}],
 ['B', 'C', {}],
 ['C', 'A', {}],
 ['C', 'B', {}],
 ['C', 'D', {}],
 ['C', 'E', {}],
 ['E', 'A', {}],
 ['E', 'C', {}],
 ['E', 'D', {}],
 ['D', 'C', {}],
 ['D', 'E', {}]]

In [ ]:
@add_method(Graph)
def number_of_nodes(self):
    return len(self._nodes)

In [ ]:
G.number_of_nodes()

5

In [ ]:
G._nodes

{'A': {}, 'B': {}, 'C': {}, 'D': {}, 'E': {}}

In [ ]:
@add_method(Graph)
def degrees(self):
    deg = {}
    
    for node in self._nodes:
        if node in self._edges:
            deg[node] =  len(self._edges[node])
        else:
            deg[node] = 0
    
    return deg

In [ ]:
G = Graph()
G.add_edges_from(edge_list)

In [ ]:
G.degrees()

{'A': 3, 'B': 2, 'C': 4, 'D': 2, 'E': 3}

In [ ]:
G._edges

{'A': {'B': {}, 'C': {}, 'E': {}},
 'B': {'A': {}, 'C': {}},
 'C': {'A': {}, 'B': {}, 'D': {}, 'E': {}},
 'D': {'C': {}, 'E': {}},
 'E': {'A': {}, 'C': {}, 'D': {}}}

In [ ]:
@add_method(Graph)
def number_of_edges(self):
    n_edges = 0
    
    for node_i in self._edges:
        n_edges += len(self._edges[node_i])
    
    # If the graph is undirected, don't double count the edges
    if not self._directed:
        n_edges /= 2
    
    return n_edges

In [ ]:
@add_method(Graph)
def is_directed(self):
    return self._directed

In [ ]:
dir(nx.Graph())

In [ ]:
G._edges

{'A': {'B': {}, 'C': {}, 'E': {}},
 'B': {'A': {}, 'C': {}},
 'C': {'A': {}, 'B': {}, 'D': {}, 'E': {}},
 'D': {'C': {}, 'E': {}},
 'E': {'A': {}, 'C': {}, 'D': {}}}

In [ ]:
@add_method(Graph)
def weights(self, weight="weight", default = 1):
    w = {}

    for node_i in self._edges:
        for node_j in self._edges[node_i]:
            if weight in self._edges[node_i][node_j]:
                w[(node_i, node_j)] = self._edges[node_i][node_j][weight]
            else:
                w[(node_i, node_j)] = default
    return w

In [ ]:
G = Graph()
G.add_edges_from(edge_list)
G._edges['A']['B']['weight']=4

In [ ]:
G.weights()

{('A', 'B'): 4,
 ('A', 'C'): 1,
 ('A', 'E'): 1,
 ('B', 'A'): 4,
 ('B', 'C'): 1,
 ('C', 'A'): 1,
 ('C', 'B'): 1,
 ('C', 'D'): 1,
 ('C', 'E'): 1,
 ('D', 'C'): 1,
 ('D', 'E'): 1,
 ('E', 'A'): 1,
 ('E', 'C'): 1,
 ('E', 'D'): 1}

In [ ]:
G._edges

{'A': {'B': {'weight': 4}, 'C': {}, 'E': {}},
 'B': {'A': {'weight': 4}, 'C': {}},
 'C': {'A': {}, 'B': {}, 'D': {}, 'E': {}},
 'D': {'C': {}, 'E': {}},
 'E': {'A': {}, 'C': {}, 'D': {}}}

## Topology and Correlations

In [ ]:
@add_method(Graph)
def neighbours(self, node):
    return list(self._edges[node].keys())

In [ ]:
G = Graph()
G.add_edges_from(edge_list)
G._edges['A']['B']['weight']=4

In [ ]:
[len(G.neighbours(node)) for node in G._edges]

[3, 2, 4, 3, 2]

In [ ]:
@add_method(Graph)
def _build_distribution(data, normalize=True):
    values = data.values()
    dist = list(Counter(values).items())
    dist.sort(key=lambda x:x[0])
    dist = np.array(dist, dtype = 'float')

    if normalize:
        #norm = dist.T[1].sum()
        norm = dist.sum(axis=0)[1]
        #norm = dist[:,1].sum()

        dist.T[1] /= norm

    return dist

In [ ]:
@add_method(Graph)
def degree_distribution(self, normalize=True):
    deg = self.degrees()
    dist = Graph._build_distribution(deg, normalize)
    
    return dist

In [ ]:
G.degree_distribution()

array([[2. , 0.4],
       [3. , 0.4],
       [4. , 0.2]])

In [ ]:
dist = G.degree_distribution()

In [ ]:
dist.sum(axis=0)[1]

1.0

In [ ]:
G._edges

{'A': {'B': {'weight': 4}, 'C': {}, 'E': {}},
 'B': {'A': {'weight': 4}, 'C': {}},
 'C': {'A': {}, 'B': {}, 'D': {}, 'E': {}},
 'D': {'C': {}, 'E': {}},
 'E': {'A': {}, 'C': {}, 'D': {}}}

In [ ]:
@add_method(Graph)
def weight_distribution(self, normalize=True):
    w = self.weights()
    dist = Graph._build_distribution(w, normalize)
    return dist

In [ ]:
G = Graph()
G.add_edges_from(edge_list)
G._edges['A']['B']['weight']=4

In [ ]:
G.weight_distribution()

array([[1.        , 0.85714286],
       [4.        , 0.14285714]])

In [ ]:
@

### Lambda and sort

In [ ]:
# lambda x:x[0]

# def name(x):
#     return x[0]

test = [[1,2],
 [12,2],
 [4,2],
 [5,3],
 [1,2]]

In [ ]:
test.sort(key=lambda x:x[1])

In [ ]:
test

[[1, 2], [1, 2], [4, 2], [12, 2], [5, 3]]

In [ ]:
test_np = np.array(test)

In [ ]:
test_np

array([[ 1,  2],
       [ 1,  2],
       [ 4,  2],
       [12,  2],
       [ 5,  3]])

In [ ]:
test_np.T

array([[ 1,  1,  4, 12,  5],
       [ 2,  2,  2,  2,  3]])

In [ ]:
test_np.T[1]

array([2, 2, 2, 2, 3])

In [ ]:
test_np[1]

array([1, 2])

## new

In [ ]:
n = 1000
test_list = [i for i in range(n)]

In [ ]:
for i in test_list:
    sum_ += i

for i in test_list:
    return i == 1

# O(n)

# O(1)

test_list[10]

# O(n^2)
# O(10*n)

1000 * 1000
output = []
for i in test_list:
    for j in test_list:
        output.append(i+j)
    